In [1]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
# ========== python ==========
import os
from logging import Logger
from pathlib import Path
import gc
# noinspection PyUnresolvedReferences
from typing import List, Dict, Tuple, Optional, Union, Callable, Final, Literal, get_args
PROJECT_DIR = Path().resolve().parents[0]
os.chdir(PROJECT_DIR)

In [2]:
import torch
from utils.setup import load_param, easy_logger, get_device
from utils.torch import load_model
from run_for_KGC import main_function
from torch.utils.tensorboard import SummaryWriter
from ignite.handlers import Checkpoint
from ignite.engine import Engine, Events

In [3]:
args = load_param('saved_models/1124/01/param.pkl')
logger = easy_logger('log/test.log')
device = get_device(device_name='cpu', logger=logger)

In [4]:
args.pre_train = False
args.train_valid_test = False  #
args.only_train = True  #
args.logger = logger
args.device = device
args.batch_size = 4
args.no_use_pe = False
args.max_len=512
args.lr=0.
del args.optuna_file, args.device_name, args.pid, args.valid_interval, args.mask_percent, 
del args.study_name, args.n_trials, args.model_path, args.tensorboard_dir
logger.info(args)

2022-11-25 23:59:16 - INFO - 2272935176.py - 12 - Namespace(notebook=False, logfile='saved_models/1124/01/log.log', param_file='saved_models/1124/01/param.pkl', resume_from_checkpoint=False, checkpoint_path=None, console_level='info', pre_train=False, story_special_num=5, relation_special_num=5, entity_special_num=5, padding_token_e=0, cls_token_e=1, mask_token_e=2, sep_token_e=3, bos_token_e=4, padding_token_r=0, cls_token_r=1, mask_token_r=2, sep_token_r=3, bos_token_r=4, padding_token_s=0, cls_token_s=1, mask_token_s=2, sep_token_s=3, bos_token_s=4, embedding_dim=128, separate_head_and_tail=False, batch_size=4, max_len=512, mask_mask_percent=0.8, mask_random_percent=0.1, mask_nomask_percent=0.1, no_use_pe=False, epoch=5000, lr=0.0, nhead=4, num_layers=4, dim_feedforward=1028, transformer_drop=0.1, position_encoder_drop=0.1, project_dir=PosixPath('/var/www'), logger=<Logger log (DEBUG)>, device=device(type='cpu'), train_valid_test=False, only_train=True)


In [5]:
CPU: Final = 'cpu'
TRAIN: Final = 'train'
TEST: Final = 'test'
MRR: Final = 'mrr'
HIT_: Final = 'hit_'
STUDY: Final = 'study'
MODEL: Final = 'model'
OPTIMIZER: Final = 'optimizer'

TRAINER: Final = 'trainer'
EVALUATOR: Final = 'evaluator'
CHECKPOINTER: Final = 'checkpointer'
CHECKPOINTER_GOOD_LOSS: Final = 'checkpointer_good_loss'
CHECKPOINTER_LAST: Final = 'checkpointer_last'

PAD_E: Final = '<pad_e>'
CLS_E: Final = '<cls_e>'
MASK_E: Final = '<mask_e>'
SEP_E: Final = '<sep_e>'
BOS_E: Final = '<bos_e>'
PAD_R: Final = '<pad_r>'
CLS_R: Final = '<cls_r>'
MASK_R: Final = '<mask_r>'
SEP_R: Final = '<sep_r>'
BOS_R: Final = '<bos_r>'

ALL_TAIL = 'all_tail'
TRIPLE = 'triple'

LOSS: Final = 'loss'
STORY_RELATION_ENTITY = ('story', 'relation', 'entity')
STORY_LOSS: Final = 'story_loss'
RELATION_LOSS: Final = 'relation_loss'
OBJECT_LOSS: Final = 'entity_loss'
LOSS_NAME3 = (STORY_LOSS, RELATION_LOSS, OBJECT_LOSS)

STORY_PRED: Final = 'story_pred'
RELATION_PRED: Final = 'relation_pred'
ENTITY_PRED: Final = 'entity_pred'
PRED_NAME3 = (STORY_PRED, RELATION_PRED, ENTITY_PRED)

STORY_ANS: Final = 'story_ans'
RELATION_ANS: Final = 'relation_ans'
OBJECT_ANS: Final = 'object_ans'
ANS_NAME3 = (STORY_ANS, RELATION_ANS, OBJECT_ANS)

STORY_ACCURACY: Final = 'story_accuracy'
RELATION_ACCURACY: Final = 'relation_accuracy'
ENTITY_ACCURACY: Final = 'entity_accuracy'
ACCURACY_NAME3 = (STORY_ACCURACY, RELATION_ACCURACY, ENTITY_ACCURACY)

METRIC_NAMES = [LOSS, *LOSS_NAME3, *ACCURACY_NAME3]

ALL_TITLE_LIST = [
    'ACaseOfIdentity', 'AbbeyGrange', 'CrookedMan', 'DancingMen',
    'DevilsFoot', 'ResidentPatient', 'SilverBlaze', 'SpeckledBand'
]

ABOUT_KILL_WORDS: Final = ['word.predicate:kill', 'word.predicate:notKill', 'word.predicate:beKilled']

SRO_ALL_INFO_FILE = "data/processed/KGCdata/All/SRO/info.hdf5"
SRO_ALL_TRAIN_FILE = "data/processed/KGCdata/All/SRO/train.hdf5"

CHECKPOINT_DIR: Final = 'saved_models/.tmp/check-point.{}'
MOST_GOOD_CHECKPOINT_PATH: Final = '{}/most_good/'
LATEST_CHECKPOINT_PATH: Final = '{}/most_good/'

SEED: Final = 42

In [28]:
trainer = Engine(lambda *_: None)
model, data_dict = main_function(args, logger=logger)
opt = torch.optim.Adam(model.parameters(), lr=0.1)
to_load = {MODEL: model, OPTIMIZER: opt, TRAINER: trainer}
path_ = "saved_models/.tmp/check-point.pid=13960/latest/checkpoint_2500.pt"
Checkpoint.load_objects(to_load=to_load, checkpoint=torch.load(path_))
good_checkpoint = Checkpoint({}, '.tmp/')
to_load = {CHECKPOINTER: good_checkpoint}
path_ = "saved_models/.tmp/check-point.pid=13960/most_good/checkpoint_400_loss=-19.6442.pt"

Checkpoint.load_objects(to_load=to_load, checkpoint=torch.load(path_))

2022-11-26 00:15:40 - INFO - data_helper.py - 438 - ==========Show DataHelper==========
2022-11-26 00:15:40 - INFO - data_helper.py - 439 - ==========
2022-11-26 00:15:40 - INFO - data_helper.py - 205 - ==========Show MyRawData==========
2022-11-26 00:15:40 - INFO - data_helper.py - 206 - entity_length: 7857
2022-11-26 00:15:40 - INFO - data_helper.py - 207 - relation_length: 63
2022-11-26 00:15:40 - INFO - data_helper.py - 210 - ==========Show MyRawData==========
2022-11-26 00:15:40 - INFO - data_helper.py - 441 - ==========
2022-11-26 00:15:40 - INFO - data_helper.py - 443 - entity_special_num: 5
2022-11-26 00:15:40 - INFO - data_helper.py - 444 - relation_special_num: 5
2022-11-26 00:15:40 - INFO - data_helper.py - 445 - ==========Show DataHelper==========


In [29]:
torch.load(path_).keys()

dict_keys(['model', 'optimizer', 'trainer', 'checkpointer'])

In [37]:
to_save = {MODEL: model, OPTIMIZER: opt, TRAINER: trainer}
last_checkpoint = Checkpoint({}, '.tmp/', global_step_transform= lambda *_: 2500)
last_checkpoint.to_save = {MODEL: model, OPTIMIZER: opt, TRAINER: trainer, CHECKPOINTER_GOOD_LOSS: good_checkpoint, CHECKPOINTER: last_checkpoint}

In [38]:
last_checkpoint(trainer)

In [39]:
last_checkpoint.last_checkpoint

PosixPath('.tmp/checkpoint_2500.pt')

In [41]:
torch.load(last_checkpoint.last_checkpoint).keys()

dict_keys(['model', 'optimizer', 'trainer', 'checkpointer_good_loss', 'checkpointer'])

In [9]:
trainer.state

State:
	iteration: 32500
	epoch: 2500
	epoch_length: 13
	max_epochs: 2500
	output: <class 'NoneType'>
	batch: <class 'NoneType'>
	metrics: <class 'dict'>
	dataloader: <class 'NoneType'>
	seed: <class 'NoneType'>
	times: <class 'dict'>

In [24]:
print(good_checkpoint.Item[0])

ignite.handlers.checkpoint.Item[0]


In [11]:
triple = data_dict['triple']
data_helper = data_dict['data_helper']
entities = data_helper.processed_entities
d_e = {e: i for i, e in enumerate(processed_entities)}
relations = data_helper.processed_relations
d_r = {r: i for i, r in enumerate(processed_relations)}
[print(entities[_t[0]], relations[_t[1]], entities[_t[2]]) for _t in triple]
pass

[[   4    4    4]
 [ 206   43 7802]
 [ 206   43 7850]
 ...
 [3432   42   18]
 [3432   49 6833]
 [3432   43 5996]]
<bos_e> <bos_r> <bos_e>
AbbeyGrange:001 kgc:when AbbeyGrange:Winter_of_1897
AbbeyGrange:001 kgc:when AbbeyGrange:1897-02-01T08
AbbeyGrange:001 kgc:hasPredicate word.predicate:wakeUp
AbbeyGrange:001 kgc:time DateTime:1897-02-01T08:00:00
AbbeyGrange:001 kgc:when AbbeyGrange:early_morning
AbbeyGrange:001 kgc:whom AllTitle:Watson
AbbeyGrange:001 kgc:subject AllTitle:Holmes
<bos_e> <bos_r> <bos_e>
AbbeyGrange:002 kgc:when AbbeyGrange:1897-02-01T010
AbbeyGrange:002 kgc:hasPredicate word.predicate:takeOut
AbbeyGrange:002 kgc:time DateTime:1897-02-01T01:00:00
AbbeyGrange:002 kgc:subject AllTitle:Holmes
AbbeyGrange:002 kgc:what AbbeyGrange:letter
AbbeyGrange:002 kgc:where AbbeyGrange:train
<bos_e> <bos_r> <bos_e>
AbbeyGrange:003 kgc:subject AllTitle:Holmes
AbbeyGrange:003 kgc:what AbbeyGrange:letter
AbbeyGrange:003 kgc:hasPredicate word.predicate:readAloud
AbbeyGrange:003 kgc:what A

In [15]:
data_helper.data.train_triple.shape

(12755, 3)

In [7]:
model = load_model(model, "saved_models/1123/01/model.pth", 'cpu')
triple = data_dict['triple']
model.to('cpu')
del data_dict
torch.cuda.empty_cache()
gc.collect()
model.eval()

KgStoryTransformer01(
  (emb_entity): Embedding(7862, 128, padding_idx=0)
  (emb_relation): Embedding(131, 128, padding_idx=0)
  (head_dense): Linear(in_features=128, out_features=128, bias=True)
  (head_dense_activation): GELU(approximate=none)
  (head_dense_norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
  (pe): PositionalEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=128, out_features=128, bias=True)
        )
        (linear1): Linear(in_features=128, out_features=1028, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=1028, out_features=128, bias=True)
        (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (dropou

In [8]:
print(f"{model.weight_head.data=}")
print(f"{model.weight_relation.data=}")
print(f"{model.weight_tail.data=}")

model.weight_head.data=tensor(1.1678)
model.weight_relation.data=tensor(0.8261)
model.weight_tail.data=tensor(1.1219)


In [9]:
processed_entities = data_helper.processed_entities
[print(i, e) for i, e, in enumerate(processed_entities)]
d_e = {e: i for i, e in enumerate(processed_entities)}

0 <pad_e>
1 <cls_e>
2 <mask_e>
3 <sep_e>
4 <bos_e>
5 AllTitle:Holmes
6 AllTitle:Watson
7 ACaseOfIdentity:Woman
8 ResidentPatient:police_officer
9 CrookedMan:Monday
10 ResidentPatient:Helper_boy
11 AbbeyGrange:Residents_of_Abbey_Grange
12 AbbeyGrange:Sir_Eustace_Brackenstall
13 SilverBlaze:Silas_Brown
14 CrookedMan:dwelleers_of_mountain
15 DevilsFoot:Mortimer
16 AbbeyGrange:police
17 AbbeyGrange:Sailor
18 SpeckledBand:Roylott
19 ACaseOfIdentity:Ned
20 SilverBlaze:Gypsy
21 ACaseOfIdentity:Man
22 SpeckledBand:coroner
23 AbbeyGrange:Mary
24 AbbeyGrange:siblings_of_Lady_Brackenstall
25 DancingMen:Maid
26 SilverBlaze:Subordinates_of_Silas_Brown
27 SilverBlaze:Wasteland
28 SilverBlaze:Gypsy_of_Dartmoor
29 SpeckledBand:mother_of_sister
30 ResidentPatient:Robbery_of_the_indentation
31 AbbeyGrange:eight_maids
32 ResidentPatient:Lanner_Inspector
33 CrookedMan:guest
34 SilverBlaze:Criminal
35 ACaseOfIdentity:father_of_Sutherland
36 SpeckledBand:lieutenant_commander
37 DancingMen:Willson
38 Crooked

In [10]:
processed_relations = data_helper.processed_relations
[print(i, r) for i, r, in enumerate(processed_relations)]
d_r = {r: i for i, r in enumerate(processed_relations)}

0 <pad_r>
1 <cls_r>
2 <mask_r>
3 <sep_r>
4 <bos_r>
5 rdf:type
6 kgc:subject
7 kgc:ActionOption
8 kgc:Not
9 kgc:RelationBetweenScene
10 kgc:SceneObjectProperty
11 kgc:SceneProperty
12 kgc:TargetObjProperty
13 kgc:adjunct
14 kgc:around
15 kgc:at_the_same_time
16 kgc:because
17 kgc:can
18 kgc:canNot
19 kgc:from
20 kgc:hasPart
21 kgc:hasPredicate
22 kgc:hasProperty
23 kgc:how
24 kgc:if
25 kgc:infoReceiver
26 kgc:infoSource
27 kgc:left
28 kgc:middle
29 kgc:near
30 kgc:next_to
31 kgc:ofPart
32 kgc:ofWhole
33 kgc:on
34 kgc:opposite
35 kgc:orTarget
36 kgc:otherwise
37 kgc:right
38 kgc:subject
39 kgc:then
40 kgc:therefore
41 kgc:to
42 kgc:what
43 kgc:when
44 kgc:when_during
45 kgc:where
46 kgc:whom
47 kgc:why
48 kgc:LocationProperty
49 kgc:time
50 kgc:AbstractTime
51 kgc:Action
52 kgc:Animal
53 kgc:CanAction
54 kgc:CanNotAction
55 kgc:NotAction
56 kgc:OFobj
57 kgc:ORobj
58 kgc:Object
59 kgc:PhysicalObject
60 kgc:Person
61 kgc:Place
62 kgc:Property
63 kgc:Scene
64 kgc:Situation
65 kgc:Statement


In [11]:
for i, _triple in enumerate(triple):
    print(i, processed_entities[_triple[0]], processed_relations[_triple[1]], processed_entities[_triple[2]])

0 <bos_e> <bos_r> <bos_e>
1 AbbeyGrange:001 kgc:subject AllTitle:Holmes
2 AbbeyGrange:001 kgc:hasPredicate word.predicate:wakeUp
3 AbbeyGrange:001 kgc:time DateTime:1897-02-01T08:00:00
4 AbbeyGrange:001 kgc:when AbbeyGrange:early_morning
5 AbbeyGrange:001 kgc:whom AllTitle:Watson
6 AbbeyGrange:001 kgc:when AbbeyGrange:Winter_of_1897
7 AbbeyGrange:001 kgc:when AbbeyGrange:1897-02-01T08
8 <bos_e> <bos_r> <bos_e>
9 AbbeyGrange:002 kgc:what AbbeyGrange:letter
10 AbbeyGrange:002 kgc:hasPredicate word.predicate:takeOut
11 AbbeyGrange:002 kgc:when AbbeyGrange:1897-02-01T010
12 AbbeyGrange:002 kgc:time DateTime:1897-02-01T01:00:00
13 AbbeyGrange:002 kgc:subject AllTitle:Holmes
14 AbbeyGrange:002 kgc:where AbbeyGrange:train
15 <bos_e> <bos_r> <bos_e>
16 AbbeyGrange:003 kgc:hasPredicate word.predicate:readAloud
17 AbbeyGrange:003 kgc:what AbbeyGrange:004
18 AbbeyGrange:003 kgc:subject AllTitle:Holmes
19 AbbeyGrange:003 kgc:what AbbeyGrange:007
20 AbbeyGrange:003 kgc:time DateTime:1897-02-01T01:0

In [12]:
story_entities = [processed_entities[_n] for  _n in triple[:,0]]
story_entities

['<bos_e>',
 'AbbeyGrange:001',
 'AbbeyGrange:001',
 'AbbeyGrange:001',
 'AbbeyGrange:001',
 'AbbeyGrange:001',
 'AbbeyGrange:001',
 'AbbeyGrange:001',
 '<bos_e>',
 'AbbeyGrange:002',
 'AbbeyGrange:002',
 'AbbeyGrange:002',
 'AbbeyGrange:002',
 'AbbeyGrange:002',
 'AbbeyGrange:002',
 '<bos_e>',
 'AbbeyGrange:003',
 'AbbeyGrange:003',
 'AbbeyGrange:003',
 'AbbeyGrange:003',
 'AbbeyGrange:003',
 'AbbeyGrange:003',
 'AbbeyGrange:003',
 'AbbeyGrange:003',
 'AbbeyGrange:003',
 '<bos_e>',
 'AbbeyGrange:004',
 'AbbeyGrange:004',
 'AbbeyGrange:004',
 '<bos_e>',
 'AbbeyGrange:005',
 'AbbeyGrange:005',
 'AbbeyGrange:005',
 'AbbeyGrange:005',
 '<bos_e>',
 'AbbeyGrange:006',
 'AbbeyGrange:006',
 '<bos_e>',
 'AbbeyGrange:007',
 'AbbeyGrange:007',
 'AbbeyGrange:007',
 'AbbeyGrange:007',
 '<bos_e>',
 'AbbeyGrange:008',
 'AbbeyGrange:008',
 'AbbeyGrange:008',
 'AbbeyGrange:008',
 'AbbeyGrange:008',
 '<bos_e>',
 'AbbeyGrange:009',
 'AbbeyGrange:009',
 '<bos_e>',
 'AbbeyGrange:009a',
 'AbbeyGrange:009a'

In [13]:
model.transformer.layers

ModuleList(
  (0): TransformerEncoderLayer(
    (self_attn): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=128, out_features=128, bias=True)
    )
    (linear1): Linear(in_features=128, out_features=1028, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
    (linear2): Linear(in_features=1028, out_features=128, bias=True)
    (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
    (norm2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
    (dropout1): Dropout(p=0.1, inplace=False)
    (dropout2): Dropout(p=0.1, inplace=False)
    (activation): GELU(approximate=none)
  )
  (1): TransformerEncoderLayer(
    (self_attn): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=128, out_features=128, bias=True)
    )
    (linear1): Linear(in_features=128, out_features=1028, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
    (linear2): Linear(in_features=1028, out_features=128, bias=True)
    (norm

In [42]:
with torch.no_grad():
    _start_index = story_entities.index('SpeckledBand:101')-1 
    _end_index = story_entities.index('SpeckledBand:165')-1 
    question_ = torch.tensor(
        [
            [d_e['<bos_e>'],d_r['<bos_r>'],d_e['<bos_e>']],
            [d_e['<mask_e>'],d_r['kgc:hasPredicate'],d_e['word.predicate:kill']],
            [d_e['<mask_e>'],d_r['kgc:whom'],d_e['SpeckledBand:Julia']],
            [d_e['<mask_e>'],d_r['kgc:subject'],d_e['<mask_e>']],
            [d_e['<mask_e>'],d_r['kgc:why'],d_e['<mask_e>']],
            [d_e['<mask_e>'],d_r['kgc:what'],d_e['<mask_e>']],
            [d_e['<mask_e>'],d_r['kgc:where'],d_e['<mask_e>']],
        ]
    )
    mask_ = torch.tensor(
        [
            [False, False, True],
            [True, False, True],
            [True, False, True],
            [True, False, True],
            [True, False, True],
            [True, False, True],
            [True, False, True],
        ]
    )
    
    print(_start_index, _end_index)
    
    questions = torch.cat([torch.from_numpy(triple[_start_index: _end_index]), question_], dim=0).unsqueeze(0)
    masks = torch.cat([torch.zeros(_end_index-_start_index,3), mask_], dim=0).to(torch.bool).transpose(1,0).unsqueeze(0)
    
    for i, n in  enumerate(torch.bincount(questions[0][:,2], minlength=len(processed_entities))):
        if n>0: print('count', processed_entities[i], n.item())

    _, story_pred, relation_pred, entity_pred = model.pre_train_forward(questions, masks[:,0], masks[:,1], masks[:,2])
    sorted_ = torch.argsort(entity_pred.to('cpu'), dim=1, descending=True)
    for i in range(100):
        ans_= sorted_[:, i]
        a_, b_, c_, subj_, why_, what_, where_ = ans_
        # print(processed_entities[a_], processed_entities[b_], processed_entities[c_])
        print(f"rank{i}, {processed_entities[subj_]}, {processed_entities[why_]}, "
              f"{processed_entities[what_]}, {processed_entities[where_]}")
    

14215 14591
count <mask_e> 4
count <bos_e> 66
count AllTitle:Holmes 10
count AllTitle:Watson 3
count SpeckledBand:Roylott 14
count SpeckledBand:coroner 1
count SpeckledBand:Percy_Armitage 3
count SpeckledBand:Helen 22
count SpeckledBand:Julia 6
count SpeckledBand:sister 1
count SpeckledBand:Roma 2
count SpeckledBand:112 1
count SpeckledBand:130 1
count SpeckledBand:132 1
count SpeckledBand:137 1
count SpeckledBand:138 1
count SpeckledBand:139 1
count SpeckledBand:140 1
count SpeckledBand:141 1
count SpeckledBand:149a 1
count word.predicate:visit 1
count word.predicate:exist 4
count word.predicate:getOff 1
count word.predicate:live 1
count word.predicate:come 1
count word.predicate:beBroken 1
count word.predicate:propose 2
count word.predicate:calculate 1
count word.predicate:trace 1
count word.predicate:leave 3
count word.predicate:put 1
count word.predicate:turnOn 1
count word.predicate:have 2
count word.predicate:call 1
count word.predicate:accept 1
count word.predicate:explain 1
cou

In [19]:
with torch.no_grad():
    _start_index = story_entities.index('DevilsFoot:301')-1 
    _end_index = story_entities.index('DevilsFoot:366')-1 
    question_ = torch.tensor(
        [
            [d_e['<bos_e>'],d_r['<bos_r>'],d_e['<bos_e>']],
            [d_e['<mask_e>'],d_r['kgc:hasPredicate'],d_e['word.predicate:kill']],
            [d_e['<mask_e>'],d_r['kgc:whom'],d_e['DevilsFoot:Brenda']],
            [d_e['<mask_e>'],d_r['kgc:subject'],d_e['<mask_e>']],
            [d_e['<mask_e>'],d_r['kgc:why'],d_e['<mask_e>']],
            [d_e['<mask_e>'],d_r['kgc:what'],d_e['<mask_e>']],
            [d_e['<mask_e>'],d_r['kgc:where'],d_e['<mask_e>']],
        ]
    )
    mask_ = torch.tensor(
        [
            [False, False, False],
            [True, False, False],
            [True, False, False],
            [True, False, True],
            [True, False, True],
            [True, False, True],
            [True, False, True],
        ]
    )
    
    print(_start_index, _end_index)
    
    questions = torch.cat([torch.from_numpy(triple[_start_index: _end_index]), question_], dim=0).unsqueeze(0)
    masks = torch.cat([torch.zeros(_end_index-_start_index,3), mask_], dim=0).to(torch.bool).transpose(1,0).unsqueeze(0)

    _, story_pred, relation_pred, entity_pred = model.pre_train_forward(questions, masks[:,0], masks[:,1], masks[:,2])
    sorted_ = torch.argsort(entity_pred.to('cpu'), dim=1, descending=True)
    for i in range(100):
        ans_= sorted_[:, i]
        subj_, why_, what_, where_ = ans_
        print(f"rank{i}, {processed_entities[subj_]}, {processed_entities[why_]}, "
              f"{processed_entities[what_]}, {processed_entities[where_]}")

8971 9286
rank0, DevilsFoot:Mortimer, <bos_e>, <bos_e>, DevilsFoot:living_room
rank1, AllTitle:Holmes, word.predicate:say, word.predicate:say, SilverBlaze:stable
rank2, AllTitle:Watson, AbbeyGrange:Lady_Brackenstall, CrookedMan:Nancy, word.predicate:notice
rank3, DevilsFoot:George, word.predicate:die, DevilsFoot:living_room, word.predicate:handOver
rank4, DevilsFoot:Owen, DevilsFoot:George, AbbeyGrange:Lady_Brackenstall, ACaseOfIdentity:head_of_Hosmer
rank5, DevilsFoot:Porter, CrookedMan:Nancy, word.predicate:die, word.predicate:say
rank6, DevilsFoot:Brenda, word.predicate:handOver, DevilsFoot:Case_1, AbbeyGrange:1897-01-31T22:30
rank7, DevilsFoot:window, DevilsFoot:living_room, word.predicate:lock, word.predicate:die
rank8, DevilsFoot:Doctor_Richard, word.predicate:think, word.predicate:arrive, ACaseOfIdentity:100_GBP
rank9, SilverBlaze:362, DevilsFoot:Reproduction_of_lamp, DancingMen:078, word.predicate:feel
rank10, word.predicate:send, DevilsFoot:Devil, word.predicate:wear, Resident

In [22]:
with torch.no_grad():
    _start_index = story_entities.index('DevilsFoot:301')-1 
    _end_index = story_entities.index('DevilsFoot:366')-1 
    question_ = torch.tensor(
        [
            [d_e['<bos_e>'],d_r['<bos_r>'],d_e['<bos_e>']],
            [d_e['<mask_e>'],d_r['kgc:hasPredicate'],d_e['word.predicate:kill']],
            [d_e['<mask_e>'],d_r['kgc:whom'],d_e['DevilsFoot:Mortimer']],
            [d_e['<mask_e>'],d_r['kgc:subject'],d_e['<mask_e>']],
            [d_e['<mask_e>'],d_r['kgc:why'],d_e['<mask_e>']],
            [d_e['<mask_e>'],d_r['kgc:what'],d_e['<mask_e>']],
            [d_e['<mask_e>'],d_r['kgc:where'],d_e['<mask_e>']],
        ]
    )
    mask_ = torch.tensor(
        [
            [False, False, False],
            [True, False, False],
            [True, False, False],
            [True, False, True],
            [True, False, True],
            [True, False, True],
            [True, False, True],
        ]
    )
    
    print(_start_index, _end_index)
    
    questions = torch.cat([torch.from_numpy(triple[_start_index: _end_index]), question_], dim=0).unsqueeze(0)
    masks = torch.cat([torch.zeros(_end_index-_start_index,3), mask_], dim=0).to(torch.bool).transpose(1,0).unsqueeze(0)

    _, story_pred, relation_pred, entity_pred = model.pre_train_forward(questions, masks[:,0], masks[:,1], masks[:,2])
    sorted_ = torch.argsort(entity_pred.to('cpu'), dim=1, descending=True)
    for i in range(100):
        ans_= sorted_[:, i]
        subj_, why_, what_, where_ = ans_
        print(f"rank{i}, {processed_entities[subj_]}, {processed_entities[why_]}, "
              f"{processed_entities[what_]}, {processed_entities[where_]}")

8971 9286
rank0, DevilsFoot:Mortimer, <bos_e>, <bos_e>, DevilsFoot:living_room
rank1, AllTitle:Holmes, word.predicate:say, word.predicate:say, word.predicate:notice
rank2, AllTitle:Watson, AbbeyGrange:Lady_Brackenstall, CrookedMan:Nancy, SilverBlaze:stable
rank3, DevilsFoot:George, word.predicate:die, DevilsFoot:living_room, word.predicate:handOver
rank4, DevilsFoot:Owen, DevilsFoot:George, AbbeyGrange:Lady_Brackenstall, ACaseOfIdentity:head_of_Hosmer
rank5, DevilsFoot:Porter, word.predicate:handOver, word.predicate:die, AbbeyGrange:1897-01-31T22:30
rank6, DevilsFoot:Brenda, DevilsFoot:living_room, DevilsFoot:Case_1, word.predicate:say
rank7, DevilsFoot:window, CrookedMan:Nancy, word.predicate:lock, word.predicate:die
rank8, SilverBlaze:362, word.predicate:think, word.predicate:arrive, ACaseOfIdentity:100_GBP
rank9, DevilsFoot:Doctor_Richard, DevilsFoot:Reproduction_of_lamp, DancingMen:078, word.predicate:feel
rank10, word.predicate:send, DevilsFoot:Devil, ResidentPatient:304, Resident

In [23]:
with torch.no_grad():
    _start_index = story_entities.index('ResidentPatient:182')-1 
    _end_index = story_entities.index('ResidentPatient:244')-1 
    question_ = torch.tensor(
        [
            [d_e['<bos_e>'],d_r['<bos_r>'],d_e['<bos_e>']],
            [d_e['<mask_e>'],d_r['kgc:hasPredicate'],d_e['word.predicate:kill']],
            [d_e['<mask_e>'],d_r['kgc:whom'],d_e['ResidentPatient:Blessington']],
            [d_e['<mask_e>'],d_r['kgc:subject'],d_e['<mask_e>']],
            [d_e['<mask_e>'],d_r['kgc:why'],d_e['<mask_e>']],
            [d_e['<mask_e>'],d_r['kgc:what'],d_e['<mask_e>']],
            [d_e['<mask_e>'],d_r['kgc:where'],d_e['<mask_e>']],
        ]
    )
    mask_ = torch.tensor(
        [
            [False, False, False],
            [True, False, False],
            [True, False, False],
            [True, False, True],
            [True, False, True],
            [True, False, True],
            [True, False, True],
        ]
    )
    
    print(_start_index, _end_index)
    
    questions = torch.cat([torch.from_numpy(triple[_start_index: _end_index]), question_], dim=0).unsqueeze(0)
    masks = torch.cat([torch.zeros(_end_index-_start_index,3), mask_], dim=0).to(torch.bool).transpose(1,0).unsqueeze(0)

    _, story_pred, relation_pred, entity_pred = model.pre_train_forward(questions, masks[:,0], masks[:,1], masks[:,2])
    sorted_ = torch.argsort(entity_pred.to('cpu'), dim=1, descending=True)
    for i in range(100):
        ans_= sorted_[:, i]
        subj_, why_, what_, where_ = ans_
        print(f"rank{i}, {processed_entities[subj_]}, {processed_entities[why_]}, "
              f"{processed_entities[what_]}, {processed_entities[where_]}")

10871 11177
rank0, ResidentPatient:Blessington, <bos_e>, AllTitle:Holmes, ResidentPatient:Examination_room
rank1, AllTitle:Holmes, word.predicate:walk, <bos_e>, <bos_e>
rank2, ResidentPatient:Percy_Trevelyan, word.predicate:notExist, word.predicate:notExist, ResidentPatient:Blessington
rank3, AllTitle:Watson, word.predicate:find, AllTitle:Watson, AllTitle:Holmes
rank4, ResidentPatient:Elderly_man, AllTitle:Holmes, CrookedMan:Barclay, ResidentPatient:Helper_boy
rank5, ResidentPatient:Young_man, AbbeyGrange:Lady_Brackenstall, ResidentPatient:Blessington, SpeckledBand:1883-04-02T04
rank6, ResidentPatient:Lanner_Inspector, DevilsFoot:Mortimer, SpeckledBand:the_day_Julia_died, DancingMen:Dancing_dolls_A
rank7, word.predicate:hear, ResidentPatient:Blessington, AbbeyGrange:041, DancingMen:window_of_study_room
rank8, word.predicate:approach, CrookedMan:Nancy, word.predicate:say, SpeckledBand:milk
rank9, word.predicate:examine, ACaseOfIdentity:Purple_plush, CrookedMan:Nancy, word.predicate:visi

In [27]:
with torch.no_grad():
    _start_index = story_entities.index('SilverBlaze:230')-1 
    _end_index = story_entities.index('SilverBlaze:298')-1 
    question_ = torch.tensor(
        [
            [d_e['<bos_e>'],d_r['<bos_r>'],d_e['<bos_e>']],
            [d_e['<mask_e>'],d_r['kgc:hasPredicate'],d_e['word.predicate:take']],
            [d_e['<mask_e>'],d_r['kgc:whom'],d_e['SilverBlaze:Silver_Blaze']],
            [d_e['<mask_e>'],d_r['kgc:subject'],d_e['<mask_e>']],
            [d_e['<mask_e>'],d_r['kgc:why'],d_e['<mask_e>']],
            [d_e['<mask_e>'],d_r['kgc:what'],d_e['<mask_e>']],
            [d_e['<mask_e>'],d_r['kgc:where'],d_e['<mask_e>']],
        ]
    )
    mask_ = torch.tensor(
        [
            [False, False, False],
            [True, False, False],
            [True, False, False],
            [True, False, True],
            [True, False, True],
            [True, False, True],
            [True, False, True],
        ]
    )
    
    print(_start_index, _end_index)
    
    questions = torch.cat([torch.from_numpy(triple[_start_index: _end_index]), question_], dim=0).unsqueeze(0)
    masks = torch.cat([torch.zeros(_end_index-_start_index,3), mask_], dim=0).to(torch.bool).transpose(1,0).unsqueeze(0)

    _, story_pred, relation_pred, entity_pred = model.pre_train_forward(questions, masks[:,0], masks[:,1], masks[:,2])
    sorted_ = torch.argsort(entity_pred.to('cpu'), dim=1, descending=True)
    for i in range(100):
        ans_= sorted_[:, i]
        subj_, why_, what_, where_ = ans_
        print(f"rank{i}, {processed_entities[subj_]}, {processed_entities[why_]}, "
              f"{processed_entities[what_]}, {processed_entities[where_]}")

12783 13114
rank0, AllTitle:Holmes, <bos_e>, <bos_e>, DevilsFoot:living_room
rank1, SilverBlaze:Silas_Brown, AbbeyGrange:Lady_Brackenstall, SilverBlaze:Silver_Blaze, word.predicate:notice
rank2, AllTitle:Watson, word.predicate:say, word.predicate:say, SilverBlaze:stable
rank3, SilverBlaze:Silver_Blaze, word.predicate:examine, AllTitle:Holmes, DevilsFoot:cottage_of_Pordeux_Bay
rank4, SilverBlaze:Inspector_Gregory, SilverBlaze:Silver_Blaze, word.predicate:ask, AllTitle:Holmes
rank5, SilverBlaze:John_Straker, word.predicate:find, word.predicate:wear, <bos_e>
rank6, ACaseOfIdentity:door, DevilsFoot:Case_2, word.predicate:notExist, SilverBlaze:Silas_Brown
rank7, SilverBlaze:Traces_of_man_s_shoes, SilverBlaze:094, DevilsFoot:Case_2, word.predicate:bring
rank8, SilverBlaze:Stableman, word.predicate:exist, AbbeyGrange:Lady_Brackenstall, CrookedMan:Royal_Mallows
rank9, SilverBlaze:Wife_of_John_Straker, word.predicate:important, SpeckledBand:safe, word.predicate:disappear
rank10, word.predicate:

In [ ]:
with torch.no_grad():
     _start_index = story_entities.index('SilverBlaze:230')-1 
    _end_index = story_entities.index('SilverBlaze:298')-1 
    question_ = torch.tensor(
        [
            [d_e['<bos_e>'],d_r['<bos_r>'],d_e['<bos_e>']],
            [d_e['<mask_e>'],d_r['kgc:hasPredicate'],d_e['<mask_e>']],
            [d_e['<mask_e>'],d_r['kgc:whom'],d_e['SilverBlaze:Silver_Blaze']],
            [d_e['<mask_e>'],d_r['kgc:subject'],d_e['<mask_e>']],
            [d_e['<mask_e>'],d_r['kgc:why'],d_e['<mask_e>']],
            [d_e['<mask_e>'],d_r['kgc:what'],d_e['<mask_e>']],
            [d_e['<mask_e>'],d_r['kgc:where'],d_e['<mask_e>']],
        ]
    )
    mask_ = torch.tensor(
        [
            [False, False, False],
            [True, False, True],
            [True, False, False],
            [True, False, True],
            [True, False, True],
            [True, False, True],
            [True, False, True],
            [False, False, False],
        ]
    )
    
    print(_start_index, _end_index)
    
    questions = torch.cat([torch.from_numpy(triple[_start_index: _end_index]), question_], dim=0).unsqueeze(0)
    masks = torch.cat([torch.zeros(_end_index-_start_index,3), mask_], dim=0).to(torch.bool).transpose(1,0).unsqueeze(0)

    _, story_pred, relation_pred, entity_pred = model.pre_train_forward(questions, masks[:,0], masks[:,1], masks[:,2])
    sorted_ = torch.argsort(entity_pred.to('cpu'), dim=1, descending=True)
    for i in range(100):
        ans_= sorted_[:, i]
        subj_, why_, what_, where_ = ans_
        print(f"rank{i}, {processed_entities[subj_]}, {processed_entities[why_]}, "
              f"{processed_entities[what_]}, {processed_entities[where_]}")

In [15]:
summay_writter = SummaryWriter('saved_models/1120/01/tensorboard')

In [15]:
summay_writter.add_embedding(model.emb_entity.weight.data[6:3435], metadata=processed_entities[6:3435], 
                             tag='character_and_stories', global_step=11210700)
summay_writter.flush()

In [16]:
summay_writter.add_embedding(model.emb_entity.weight.data[6:3435], metadata=processed_entities[6:3435], 
                             tag='character_and_stories', global_step=11210715)
summay_writter.flush()

In [16]:
summay_writter.add_embedding(model.emb_relation.weight.data[:], metadata=processed_relations[:], 
                             tag='relations', global_step=11210730)
summay_writter.flush()

In [45]:
summay_writter.close()